# Trimming vs Timing: Understanding Source Time and Timeline Time
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/video-db/videodb-cookbook/blob/main/editor/feature/trimming_vs_timing.ipynb)


---

This notebook explores the **"double start" concept** in Editor.

When you work with video clips, there are two separate timing controls that work independently:
- **`VideoAsset(start=...)`** controls which part of the source video to use (trimming)
- **`track.add_clip(start=...)`** controls when the clip appears in your final video (timing)

By rendering real examples, you'll see how these two parameters give you precise control over video editing. We'll use the same video throughout, changing only the trim points and timeline positions to make the behavior crystal clear.

---
## 📦 Step 1: Install dependencies

Lets install the VideoDB SDK

In [1]:
%pip -q install videodb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.3/43.3 kB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


---
## 📦 Step 2: Connect to VideoDB

Run the next cell and enter your `VIDEO_DB_API_KEY` when prompted. Your input will be masked.

In [2]:
import videodb
import os
from getpass import getpass

api_key = getpass("Please enter your VideoDB API Key: ")

os.environ["VIDEO_DB_API_KEY"] = api_key

conn = videodb.connect()

print("Connected to VideoDB securely!")

Please enter your VideoDB API Key: ··········
Connected to VideoDB securely!


---
## 📦 Step 3: Get collection and upload video

A **collection** is where your uploaded assets live. We'll upload one video and reuse it throughout this notebook to demonstrate the trimming and timing concepts.

> Tip: When you re-run this notebook, you can skip re-uploading and fetch an existing asset by ID instead (see the commented snippet below).

In [3]:
coll = conn.get_collection()

video = coll.upload(url="https://www.youtube.com/watch?v=HAX2RuZm-Fk")
print("Video uploaded:", video.id)

# Optional re-run pattern (don't upload again):
# video = coll.get_video("")

Video uploaded: m-z-019b6e0e-d801-7763-9931-e914b7f35ba1


---
## 📦 Step 4: Import Editor components

We'll use a small set of Editor objects:
- `Timeline`: the global canvas
- `Track`: a layer on the timeline
- `Clip`: a container that controls how an asset appears
- `VideoAsset`: references your uploaded video by `id`

In [4]:
from videodb import play_stream
from videodb.editor import Timeline, Track, Clip, VideoAsset

BACKGROUND = "#2B2B2B"

---
## Understanding the "Double Start"

Before we dive into examples, let's understand the core concept.

When you create a clip in Editor, there are **two independent "start" parameters** that control different things:

### 1. VideoAsset(start=X) — Source Trimming
This controls **which part of the source video** you want to use.
- `start=0` means use the video from the beginning
- `start=10` means skip the first 10 seconds of the source video
- `start=30` means skip the first 30 seconds

Think of it as: *"Where do I want to start reading from the original video file?"*

### 2. track.add_clip(start=Y, clip) — Timeline Positioning
This controls **when the clip appears** in your final video.
- `start=0` means the clip appears immediately at the beginning of your timeline
- `start=5` means the clip appears 5 seconds into your final video
- `start=10` means there's 10 seconds of empty space before the clip starts

Think of it as: *"When should this clip play in the output video?"*

### They Work Independently
The key insight is that these two parameters are **completely separate**. You can:
- Trim different parts of the source without changing timeline position
- Change timeline position without changing what part of the source you're using
- Combine both for precise control over what plays and when

Let's see this in action with real examples.

---
## 📦 Part 1: Exploring Trimming (VideoAsset start)

In this section, we'll create three clips that all appear at the **same timeline position** (position 0), but use **different parts of the source video**.

This isolates the trimming behavior so you can see exactly how `VideoAsset(start=...)` selects different segments of your source video.

Each clip will:
- Play for 10 seconds
- Start immediately on the timeline (position 0)
- Use a different starting point in the source video

Run the next cells one by one and watch how the content changes.

### Trim from beginning: start=0

This uses the first 10 seconds of the source video (0:00 to 0:010).

In [6]:
timeline = Timeline(conn)
timeline.background = BACKGROUND

clip = Clip(asset=VideoAsset(id=video.id, start=0), duration=10)

track = Track()
track.add_clip(0, clip)
timeline.add_track(track)

stream = timeline.generate_stream()
print(stream)
play_stream(stream)

https://play.videodb.io/v1/797da738-f56b-48b3-a415-fab4621c8912.m3u8


### Trim from 10 seconds: start=10

This skips the first 10 seconds of the source and uses seconds 10-20. Notice how the content is completely different, but it still appears immediately on the timeline (no delay).

In [7]:
timeline = Timeline(conn)
timeline.background = BACKGROUND

clip = Clip(asset=VideoAsset(id=video.id, start=10), duration=10)

track = Track()
track.add_clip(0, clip)
timeline.add_track(track)

stream = timeline.generate_stream()
print(stream)
play_stream(stream)

https://play.videodb.io/v1/e7aed706-0bcc-4756-b6e1-191a1b65a056.m3u8


### Trim from 20 seconds: start=20

This skips the first 20 seconds of the source and uses seconds 20-30. Again, completely different content, still appearing immediately.

In [8]:
timeline = Timeline(conn)
timeline.background = BACKGROUND

clip = Clip(asset=VideoAsset(id=video.id, start=20), duration=10)

track = Track()
track.add_clip(0, clip)
timeline.add_track(track)

stream = timeline.generate_stream()
print(stream)
play_stream(stream)

https://play.videodb.io/v1/868c7c7c-d0bc-4364-bc7d-b00e080441c9.m3u8


**What did we just see?**

All three clips appeared at the same time on the timeline (immediately at 0 seconds), but each showed different content from the source video. The `VideoAsset(start=...)` parameter controlled which segment we extracted, without affecting when it plays in the final output.

---
## 📦 Part 2: Exploring Timing (track.add_clip start)

Now let's do the opposite. We'll use the **exact same source content** (the first 10 seconds of the video), but place it at **different positions on the timeline**.

This isolates the timing behavior so you can see exactly how `track.add_clip(start=...)` controls when your clip appears in the final video.

Each clip will:
- Use the same source content (0:00 to 0:10 from the source)
- Play for 10 seconds
- Appear at a different position on the timeline

Watch how the timing changes create empty space before the clip.

### Timeline position 0: Immediate playback

The clip appears right away. No empty space before it.

In [9]:
timeline = Timeline(conn)
timeline.background = BACKGROUND

clip = Clip(asset=VideoAsset(id=video.id, start=0), duration=10)

track = Track()
track.add_clip(0, clip)
timeline.add_track(track)

stream = timeline.generate_stream()
print(stream)
play_stream(stream)

https://play.videodb.io/v1/94ce13a4-2cd0-4658-8f6d-b976f02c6b90.m3u8


### Timeline position 5: Delayed by 5 seconds

Same content as above, but now it doesn't appear until 5 seconds into the timeline. You'll see 5 seconds of gray space first.

In [10]:
timeline = Timeline(conn)
timeline.background = BACKGROUND

clip = Clip(asset=VideoAsset(id=video.id, start=0), duration=10)

track = Track()
track.add_clip(5, clip)
timeline.add_track(track)

stream = timeline.generate_stream()
print(stream)
play_stream(stream)

https://play.videodb.io/v1/0b6c2017-7363-45d0-ae79-02f41bb7e0f8.m3u8


### Timeline position 10: Delayed by 10 seconds

Same content again, but now positioned at 10 seconds. You'll see 10 seconds of empty space before the clip plays.

In [11]:
timeline = Timeline(conn)
timeline.background = BACKGROUND

clip = Clip(asset=VideoAsset(id=video.id, start=0), duration=10)

track = Track()
track.add_clip(10, clip)
timeline.add_track(track)

stream = timeline.generate_stream()
print(stream)
play_stream(stream)

https://play.videodb.io/v1/ab09bc2c-efd3-4814-b255-b8599399622d.m3u8


**What did we just see?**

All three clips showed the exact same content (the first 10 seconds of the source video), but they appeared at different times in the final output. The `track.add_clip(start=...)` parameter controlled the timing, without affecting which part of the source video was used.

---
## 📦 Part 3: Combining Both (The Real Power)

Now that you understand trimming and timing separately, let's combine them. This is where Editor's "double start" system becomes truly powerful.

By controlling both parameters independently, you can:
- Extract specific segments from anywhere in your source video
- Position them precisely on your timeline
- Create complex edits like highlight reels, compilations, and remixes

Let's look at three examples that use both parameters together.

### Example 1: Later content, delayed playback

This takes content from 10-20 seconds of the source (`start=10`, `duration=10`), but places it at 5 seconds on the timeline. You'll see 5 seconds of empty space, then 10 seconds of content from the middle of the source video.

In [12]:
timeline = Timeline(conn)
timeline.background = BACKGROUND

clip = Clip(asset=VideoAsset(id=video.id, start=10), duration=10)

track = Track()
track.add_clip(5, clip)
timeline.add_track(track)

stream = timeline.generate_stream()
print(stream)
play_stream(stream)

https://play.videodb.io/v1/44b2a5aa-21e0-4c5b-be31-a77a87d0c89d.m3u8


### Example 2: Late content, early position

This takes content from 20-30 seconds of the source (`start=20`, `duration=10`), but places it at 3 seconds on the timeline. The final video has 3 seconds of space, then shows content from late in the source.

In [13]:
timeline = Timeline(conn)
timeline.background = BACKGROUND

clip = Clip(asset=VideoAsset(id=video.id, start=20), duration=10)

track = Track()
track.add_clip(3, clip)
timeline.add_track(track)

stream = timeline.generate_stream()
print(stream)
play_stream(stream)

https://play.videodb.io/v1/2a6ecb1c-bdcf-434f-9146-2bc2638d98cf.m3u8


### Example 3: Early content, late position

This takes content from early in the source (5-13 seconds with `start=5`, `duration=8`), but doesn't show it until 10 seconds into the timeline. The final video has 10 seconds of empty space, then shows the early content.

In [14]:
timeline = Timeline(conn)
timeline.background = BACKGROUND

clip = Clip(asset=VideoAsset(id=video.id, start=5), duration=8)

track = Track()
track.add_clip(10, clip)
timeline.add_track(track)

stream = timeline.generate_stream()
print(stream)
play_stream(stream)

https://play.videodb.io/v1/2d004a9d-1e64-47f6-9c54-293fa576addc.m3u8


**What did we just see?**

By combining both parameters, we have complete control over:
1. Which segment of the source video to use (trimming)
2. When that segment appears in the final video (timing)

This is the foundation for creating highlights reels, montages, and any complex video composition where you need precise control over your content.

---
## ✅ Wrap-up

You now understand the "double start" concept in Editor.

### The Two Controls

1. **`VideoAsset(start=X)`** — Source Trimming
   - Controls which part of the source video to use
   - `start=10` skips the first 10 seconds of your source

2. **`track.add_clip(start=Y, clip)`** — Timeline Positioning
   - Controls when the clip appears in the final video
   - `start=5` places the clip at 5 seconds on the timeline


### What You Can Do

With this understanding, you can:
- Create highlight reels by extracting specific moments
- Reuse the same video multiple times with different segments
- Build complex timelines with precise timing control
- Remix and rearrange content however you need

The key is remembering that these two parameters work independently—you control what plays and when separately.